In [1]:
from src.data_preprocessor import DataProcessor
from data_configs.configs import *
from models.neural_networks import *
from src.cross_validation import CrossValidation
import numpy as np

config = car_config
data_processor = DataProcessor(config=config)
cross_validator = CrossValidation(config=config)

In [2]:
raw_data = data_processor.load_data()
data_1 = data_processor.impute_missing_values(raw_data)
data_2 = data_processor.encode_ordinal_features(data_1)
data_3 = data_processor.standardize_data(data_2,data_2,features=['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety'])

c:\Users\Srikanta\Documents\Intro to Machine Learning\programming_assignment_1\src\data_preprocessor.py:47: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data[column].fillna(data[column].mode()[0], inplace=True)


In [15]:
raw_data

,buying,maint,doors,persons,lug_boot,safety,Class
0,vhigh,vhigh,2,2,small,low,unacc
1,vhigh,vhigh,2,2,small,med,unacc
2,vhigh,vhigh,2,2,small,high,unacc
3,vhigh,vhigh,2,2,med,low,unacc
4,vhigh,vhigh,2,2,med,med,unacc
...,...,...,...,...,...,...,...
1723,low,low,5more,more,med,med,good
1724,low,low,5more,more,med,high,vgood
1725,low,low,5more,more,big,low,unacc
1726,low,low,5more,more,big,med,good


In [4]:
data_train, data_val = cross_validator.random_partition(data_3, random_state=42)

In [5]:
data_val = data_processor.encode_nominal_features(data_val)

In [6]:
data_val

,buying,maint,doors,persons,lug_boot,safety,Class_acc,Class_good,Class_unacc,Class_vgood
599,-0.447084,-0.447084,0.447084,-1.22439,0.00000,1.22439,0,0,1,0
1201,0.447084,1.341253,-1.341253,0.00000,0.00000,0.00000,1,0,0,0
628,-0.447084,-0.447084,1.341253,-1.22439,1.22439,0.00000,0,0,1,0
1498,1.341253,-0.447084,1.341253,0.00000,0.00000,0.00000,1,0,0,0
1263,0.447084,1.341253,0.447084,1.22439,0.00000,-1.22439,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...
100,-1.341253,-1.341253,1.341253,1.22439,-1.22439,0.00000,0,0,1,0
274,-1.341253,0.447084,0.447084,-1.22439,0.00000,0.00000,0,0,1,0
1206,0.447084,1.341253,-1.341253,1.22439,-1.22439,-1.22439,0,0,1,0
101,-1.341253,-1.341253,1.341253,1.22439,-1.22439,1.22439,0,0,1,0


## One Fold Cross-Validation ##

In [7]:
best_params_linear = {'lr': 0.001, 'epochs': 17000}
best_params_ffn = {'lr': 0.0001, 'epochs': 15000, 'n_hidden': 50, 'n_hidden_2': 50}
best_params_auto = {'lr': 0.0001, 'epochs': 15000, 'n_encoder': 4}
best_params_combined = {'lr': 0.001, 'epochs': 19000, 'n_hidden_2': 40}

In [16]:
linear_scores = []
ffn_scores = []
auto_scores = []
combined_scores = []

for i, (train_set, test_set) in enumerate(cross_validator.cross_validation(data_train, n_splits=2, n_repeats=1, random_state=42, stratify=True)):

    train_set = data_processor.encode_nominal_features(train_set)
    test_set = data_processor.encode_nominal_features(test_set)

    train_data = train_set.to_numpy()
    X_train = train_data[:,:-4]
    y_train = train_data[:,-4:]

    test_data = test_set.to_numpy()
    X_test = test_data[:,:-4]
    y_test = test_data[:,-4:]

    linear = LinearNetwork(config)
    _, linear_val_losses = linear.logistic_regression(X_train,y_train,X_test,y_test,epochs=best_params_linear['epochs'],lr=best_params_linear['lr'],patience=500)

    ffn = FeedForwardNetwork(config,n_input=X_train.shape[1],n_hidden_1=best_params_ffn['n_hidden'],n_hidden_2=best_params_ffn['n_hidden'],n_output=y_train.shape[1])
    _, ffn_val_losses, _ = ffn.train(X_train,y_train,X_test,y_test,epochs=best_params_ffn['epochs'],lr=best_params_ffn['lr'],patience=500)

    autoE = AutoEncoder(config,n_input=X_train.shape[1],n_encoder=best_params_auto['n_encoder'])
    auto_losses = autoE.train(X_train, max_epochs=best_params_auto['epochs'], lr=best_params_auto['lr'])
    
    combined = CombinedModel(autoE,n_hidden_2=best_params_combined['n_hidden_2'],n_output=y_test.shape[1])
    _, combined_val_losses, _ = combined.train(X_train,y_train,X_test,y_test,epochs=best_params_combined['epochs'], lr=best_params_combined['lr'],patience=500)


    linear_score = np.min(linear_val_losses)
    ffn_score = np.min(ffn_val_losses)
    auto_score = np.min(auto_losses)
    combined_score = np.min(combined_val_losses)
    
    linear_scores.append(linear_score)
    ffn_scores.append(ffn_score)
    auto_scores.append(auto_score)
    combined_scores.append(combined_score)

avg_score_linear = np.mean(linear_scores)
avg_score_ffn = np.mean(ffn_scores)
avg_score_auto = np.mean(auto_scores)
avg_score_combined = np.mean(combined_scores)

print(f"Linear Model Tested params: {best_params_linear}, Average Score: {avg_score_linear}")
print(f"FFN Model Tested params: {best_params_ffn}, Average Score: {avg_score_ffn}")
print(f"Autoencoder Model Tested params: {best_params_auto}, Average Score: {avg_score_auto}")
print(f"Combined Model Tested params: {best_params_combined}, Average Score: {avg_score_combined}")

Linear Model Tested params: {'lr': 0.001, 'epochs': 17000}, Average Score: 0.40137900672626625
FFN Model Tested params: {'lr': 0.0001, 'epochs': 15000, 'n_hidden': 50}, Average Score: 0.09169245544147045
Autoencoder Model Tested params: {'lr': 0.0001, 'epochs': 15000, 'n_encoder': 4}, Average Score: 0.30616363126136825
Combined Model Tested params: {'lr': 0.001, 'epochs': 19000, 'n_hidden_2': 40}, Average Score: 0.1448744998842669


## Forward Propagation and Back Propagation ##

### Linear Model ###

In [19]:
linear = LinearNetwork(config)

losses, val_losses = linear.logistic_regression(X_train,y_train,X_test,y_test,epochs=2,lr=best_params_linear['lr'],patience=np.inf,verbose=True)

Initial Weights:
 [[0.0048764  0.00451726 0.00033923 0.00475053]
 [0.00371172 0.00055143 0.00064923 0.00662943]
 [0.00342213 0.00584239 0.00453272 0.00475753]
 [0.00793712 0.00124215 0.00548516 0.00173043]
 [0.00246522 0.0090802  0.00472296 0.00313716]
 [0.00317706 0.00995521 0.00362567 0.00237262]
 [0.00985201 0.00304871 0.00266064 0.00859837]]
X_train * W =
 [[ 0.02492487  0.02464716  0.01422556  0.00907256]
 [ 0.03802466  0.0195632   0.01656068  0.03193592]
 [ 0.0052771  -0.00367959 -0.00125494  0.00084617]
 ...
 [ 0.02867983  0.03757274  0.02428799  0.02316255]
 [ 0.00496497 -0.0201066  -0.00858616  0.00554278]
 [ 0.0036457   0.00574914  0.00154718  0.01000391]]

Softmax(X_train * W) =
 [[0.25167662 0.25160674 0.24899821 0.24771843]
 [0.25288271 0.24825696 0.24751268 0.25134765]
 [0.25124672 0.24900643 0.24961092 0.25013593]
 ...
 [0.2500595  0.25229317 0.24896368 0.24868365]
 [0.25237495 0.24612618 0.24897805 0.25252082]
 [0.2496014  0.25012697 0.24907815 0.25119347]]

Error =
 [[

### FFN Model ###

In [10]:
ffn = FeedForwardNetwork(config,n_input=X_train.shape[1],n_hidden_1=5,n_hidden_2=5,n_output=y_train.shape[1],verbose=True)
 
metrics, val_metrics, final_metric = ffn.train(X_train,y_train,X_test,y_test,epochs=2,lr=best_params_ffn['lr'],patience=np.inf)

Hidden Layer 1 Weights:
 [[0.00772433 0.00602232 0.00098712 0.00899257 0.00106798]
 [0.00238187 0.00105419 0.00947813 0.00323022 0.00969005]
 [0.00120666 0.00386683 0.00855773 0.00884138 0.00129811]
 [0.00976555 0.00011224 0.00954554 0.00911482 0.00552604]
 [0.00083935 0.00056432 0.00240898 0.0029178  0.00442341]
 [0.00823579 0.00733896 0.00556334 0.00576833 0.00948202]]

Hidden Layer 2 Weights:
 [[0.00718173 0.00978488 0.00026664 0.00222894 0.00810236]
 [0.00315293 0.0014915  0.00568028 0.00180645 0.00448989]
 [0.00864898 0.00811809 0.00705292 0.00872068 0.00567593]
 [0.00561759 0.00260521 0.00598373 0.00119248 0.00636292]
 [0.00123757 0.00361576 0.0052665  0.00755197 0.00825498]]

Output Layer Weights:
 [[0.00635798 0.00951931 0.00284499 0.00209112]
 [0.00780937 0.00123148 0.00613805 0.00023524]
 [0.00403521 0.00100673 0.0075278  0.0070322 ]
 [0.00095391 0.00121736 0.00135965 0.00273301]
 [0.00561701 0.0094126  0.00318333 0.00540948]]

Hidden Layer 1 Biases:
 [[0. 0. 0. 0. 0.]]

Hidd

### Autoencoder Model ###

In [11]:
autoE = AutoEncoder(config,n_input=X_train.shape[1],n_encoder=best_params_auto['n_encoder'],verbose=True)
losses = autoE.train(X_train, max_epochs=2, lr=best_params_auto['lr'])

Initial Encoder Weights:
[[0.00040495 0.00728966 0.00732425 0.00679254]
 [0.00918556 0.00495604 0.00256729 0.00943121]
 [0.00338202 0.00784474 0.00308926 0.0034591 ]
 [0.00952418 0.00874715 0.00205027 0.0016668 ]
 [0.00086363 0.00244828 0.00302101 0.0039168 ]
 [0.00232631 0.00773301 0.00817539 0.00815417]]

Initial Decoder Weights:
[[0.00720015 0.00305315 0.00593426 0.00850788 0.00258921 0.00068197]
 [0.00554123 0.00900657 0.00715757 0.00220975 0.00033281 0.00041953]
 [0.00028299 0.00311497 0.00873493 0.00559643 0.00637345 0.00484705]
 [0.00976626 0.00128913 0.00317859 0.00884707 0.00669565 0.00664766]]

Initial Encoder Biases:
[[0.00040495 0.00728966 0.00732425 0.00679254]
 [0.00918556 0.00495604 0.00256729 0.00943121]
 [0.00338202 0.00784474 0.00308926 0.0034591 ]
 [0.00952418 0.00874715 0.00205027 0.0016668 ]
 [0.00086363 0.00244828 0.00302101 0.0039168 ]
 [0.00232631 0.00773301 0.00817539 0.00815417]]

Initial Decoder Biases:
[[0.00720015 0.00305315 0.00593426 0.00850788 0.00258921

In [12]:
import pandas as pd

_, Output = autoE.forward_pass(X_train)
pd.DataFrame(Output)

,0,1,2,3,4,5
0,0.008333,-0.004565,0.013849,0.017460,0.001135,0.003297
1,0.008442,-0.004564,0.013961,0.017613,0.001207,0.003379
2,0.008281,-0.004602,0.013752,0.017382,0.001127,0.003286
3,0.008221,-0.004621,0.013668,0.017296,0.001098,0.003252
4,0.008378,-0.004565,0.013882,0.017516,0.001150,0.003312
...,...,...,...,...,...,...
686,0.008342,-0.004585,0.013818,0.017462,0.001134,0.003292
687,0.008277,-0.004606,0.013743,0.017378,0.001122,0.003279
688,0.008455,-0.004550,0.013968,0.017620,0.001182,0.003348
689,0.008272,-0.004593,0.013765,0.017376,0.001130,0.003292


In [13]:
pd.DataFrame(X_train)

,0,1,2,3,4,5
0,-1.341253,-1.341253,1.341253,1.22439,1.22439,1.22439
1,1.341253,1.341253,1.341253,-1.22439,1.22439,1.22439
2,-0.447084,0.447084,0.447084,-1.22439,0.00000,0.00000
3,-0.447084,0.447084,-0.447084,-1.22439,0.00000,-1.22439
4,-0.447084,0.447084,0.447084,1.22439,-1.22439,1.22439
...,...,...,...,...,...,...
686,1.341253,1.341253,-1.341253,1.22439,-1.22439,-1.22439
687,0.447084,0.447084,-1.341253,0.00000,1.22439,-1.22439
688,0.447084,1.341253,1.341253,1.22439,1.22439,0.00000
689,1.341253,-0.447084,0.447084,-1.22439,-1.22439,0.00000


### Combined Model ###

In [20]:
combined = CombinedModel(autoE,n_hidden_2=5,n_output=y_test.shape[1],verbose=True)
metrics, val_metrics, final_metric = combined.train(X_train,y_train,X_test,y_test,epochs=2, lr=best_params_combined['lr'],patience=500)

Autoencoder Weights (Hidden Layer 1):
[[-0.19897223 -0.21660858  0.16426085  0.43285014]
 [-0.38491301  0.31715251  0.19540782  0.01999713]
 [ 0.36271531  0.30134397  0.13822726  0.56357072]
 [ 0.54346778  0.03582247  0.19582056 -0.14137119]
 [-0.0208248   0.0097908   0.78290707 -0.17904159]
 [-0.014578    0.62268923 -0.07072667 -0.0681484 ]]

Autoencoder Biases (Hidden Layer 1):
[[-0.02741079 -0.01100522 -0.01800552 -0.04949936]]

Hidden Layer 2 Initial Weights:
[[0.00214517 0.00088517 0.00516021 0.00746642 0.00363071]
 [0.00151387 0.00722286 0.00126583 0.00835262 0.00771288]
 [0.00919317 0.00858486 0.00137869 0.00024996 0.00582891]
 [0.00394396 0.00931276 0.00046398 0.00747815 0.00666105]]

Hidden Layer 2 Initial Biases:
[[0. 0. 0. 0. 0.]]

Output Layer Initial Biases:
[[0.0076336  0.00915909 0.00521226 0.00088066]
 [0.00807222 0.00778139 0.00544301 0.00120706]
 [0.00920974 0.00803674 0.00309442 0.00048366]
 [0.00674751 0.00548631 0.00661776 0.0015848 ]
 [0.00133052 0.00356454 0.0076